In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import numpy as np

In [33]:
!pip install tensorflow==2.2.0rc2

Defaulting to user installation because normal site-packages is not writeable


In [32]:
!pip install tensorflow==2.2.8

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement tensorflow==2.2.8 (from versions: 1.13.0rc1, 1.13.0rc2, 1.13.1, 1.13.2, 1.14.0rc0, 1.14.0rc1, 1.14.0, 1.15.0rc0, 1.15.0rc1, 1.15.0rc2, 1.15.0rc3, 1.15.0, 1.15.2, 2.0.0a0, 2.0.0b0, 2.0.0b1, 2.0.0rc0, 2.0.0rc1, 2.0.0rc2, 2.0.0, 2.0.1, 2.1.0rc0, 2.1.0rc1, 2.1.0rc2, 2.1.0, 2.2.0rc0, 2.2.0rc1, 2.2.0rc2)
ERROR: No matching distribution found for tensorflow==2.2.8


In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc2


In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
num_class = 5
def creat_compiled_model():
    model = Sequential()
    model.add(NASNetMobile(input_shape=(224, 224, 3), include_top=False,
                           weights='imagenet', pooling='avg'))
    model.add(Dense(num_class, activation='softmax'))
    model.layers[0].trainable = True
    model.layers[1].trainable = True
    sgd = SGD(learning_rate=0.01, momentum=0.01, nesterov=False)
    model.compile(optimizer=sgd, loss='hinge',  metrics=['accuracy'])
    return model

In [4]:
model3 = creat_compiled_model()

In [5]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
# https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit_generator
train_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)

In [7]:
train_generator = train_datagen.flow_from_directory('labelled_data', batch_size=4)

Found 99 images belonging to 5 classes.


In [8]:
model3.fit(train_generator, epochs=3, steps_per_epoch=3)

Epoch 1/3
3/3 [==============================] - 3s 921ms/step - loss: 1.1059 - accuracy: 0.2500
Epoch 2/3
3/3 [==============================] - 2s 737ms/step - loss: 1.1246 - accuracy: 0.0833
Epoch 3/3
3/3 [==============================] - 2s 784ms/step - loss: 1.1287 - accuracy: 0.0000e+00


Je remarque que lorsque j'utilise la version 2.2.0rc2 de tensorflow, le warning suivant:  WARNING:tensorflow:sample_weight modes were coerced from disparait

In [9]:
#del image
image = load_img('./labelled_data/Front/00035.jpg', target_size=(224, 224))
#print(image)
image = img_to_array(image)
#print(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
#print(image)
# cette ligne a verifier le self?
probs = model3.predict(image)[0]
pred_labels, pred_scores = [], []
print(probs)
list_of_labels = ["Back", "Discard", "Front", "Left", "Right"]
label_idx = np.argmax(probs)
label = list_of_labels[label_idx]
score = probs[label_idx]
pred_labels.append(label)
pred_scores.append(score.item())
print(pred_labels)
print(pred_scores)

[7.7625276e-03 6.5549631e-03 4.1621615e-04 5.2657235e-02 9.3260908e-01]
['Right']
[0.9326090812683105]


In [10]:
tf.debugging.enable_check_numerics()

INFO:tensorflow:Enabled check-numerics callback in thread MainThread


In [10]:
image = load_img('./labelled_data/Right/07944.jpg', target_size=(224, 224))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
print(model3.predict_classes(image)[0])
x = int(model3.predict_classes(image)[0])
print(x, type(x))
y = 5
print(y, type(y))

2
2 <class 'int'>
5 <class 'int'>


In [11]:
sgd = SGD(learning_rate=0.01, momentum=0.01, nesterov=False)
model3.compile(optimizer=sgd, loss='hinge', metrics=['accuracy'])

In [12]:
model3.save("./labelled_data/trained_model", save_format='h5')

In [20]:
del loaded_model

In [21]:
loaded_model

NameError: name 'loaded_model' is not defined

In [24]:
loaded_model = load_model("./labelled_data/trained_model")

In [25]:
#del image
image = load_img('./labelled_data/Front/00035.jpg', target_size=(224, 224))
#print(image)
image = img_to_array(image)
#print(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
#print(image)
# cette ligne a verifier le self?
probs = loaded_model.predict(image)[0]
pred_labels, pred_scores = [], []
print(probs)
list_of_labels = ["Back", "Discard", "Front", "Left", "Right"]
label_idx = np.argmax(probs)
label = list_of_labels[label_idx]
score = probs[label_idx]
pred_labels.append(label)
pred_scores.append(score.item())
print(pred_labels)
print(pred_scores)

[0.21155082 0.01376094 0.20672642 0.51296175 0.05500011]
['Left']
[0.512961745262146]


J'ai copié un modèle entrainé du conteneur vers le host, et ce modèle marche très bien, i.e. j'arrive à prédire le label des nouveau éléments sans problème. 

*** Operational MODE: single process ***
2020-04-06 09:49:34.143034: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-04-06 09:49:34.144535: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; 
dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-04-06 09:49:34.144656: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2020-04-06 09:49:38.238807: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2020-04-06 09:49:38.239027: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: UNKNOWN ERROR (303)
2020-04-06 09:49:38.239132: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (3a785cd41c35): /proc/driver/nvidia/version does not exist
2020-04-06 09:49:38.239537: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-04-06 09:49:38.253783: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2711995000 Hz
2020-04-06 09:49:38.254100: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x559c84c64b90 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-04-06 09:49:38.254182: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version

